## metrics

In [4]:
import json
import collections
import pandas as pd
import numpy as np
english_exp_names_1 = []
english_exp_names_3 = ['english', 'english_stopword', 'english_delete_5', 'english_delete_10', \
        'english_insert_5', 'english_insert_10', \
        'english_replace_5', 'english_replace_10', \
        'english_repeat_sentences', 'english_shuffle_sentences', 'english_shuffle_words', 'english_shuffle_instruction']

exact_match = collections.defaultdict(list)
rougeL = collections.defaultdict(list)
classifications = ['answerability_classification', 'cause_effect_classification', 'coreference_resolution', \
    'dialogue_act_recognition', 'word_analogy', 'textual_entailment']
non_classifications = ['data_to_text', 'grammar_error_correction', 'title_generation',\
    'keyword_tagging', 'overlap_extraction', 'question_rewriting']
for english_exp_name in english_exp_names_1:
    metric_file = open('/home/gujiashe/Tk-Instruct/tk_outputs/output_{}/all_results.json'.format(english_exp_name), 'r')
    metric = json.load(metric_file)


    for classification in classifications:
        exact_match[classification].append(metric['eval_exact_match_for_{}'.format(classification)])
    exact_match['eval_exact_match'].append(metric['eval_exact_match'])
    for non_classification in non_classifications:
        rougeL[non_classification].append(metric['eval_rougeL_for_{}'.format(non_classification)])
    rougeL['eval_rougeL'].append(metric['eval_rougeL'])

metric_file = open('/home/gujiashe/Tk-Instruct/tk_outputs/output_{}/all_results.json'.format("english"), 'r')
metric = json.load(metric_file)
metric_names = metric.keys()
def create_dict(keys):
    d = {}
    for key in keys:
        d[key] = []
    return d    
for english_exp_name in english_exp_names_3:
    metric_samples = create_dict(metric_names)
    metric_means = create_dict(metric_names)
    metric_stds = create_dict(metric_names)
    for index in [1, 2, 3]:
        _english_exp_name = english_exp_name + "_{}".format(index)

        metric_file = open('/home/gujiashe/Tk-Instruct/tk_outputs/output_{}/all_results.json'.format(_english_exp_name), 'r')
        metric = json.load(metric_file)
        for metric_name in metric_names:
            metric_samples[metric_name].append(metric[metric_name])
    
    for metric_name in metric_names:
        metric_means[metric_name] = round(np.mean(metric_samples[metric_name]), 1)
    for metric_name in metric_names:
        metric_stds[metric_name] = round(np.std(metric_samples[metric_name]), 1)

    # metrics = metric_means

    for classification in classifications:
        metric_mean = metric_means['eval_exact_match_for_{}'.format(classification)]
        metric_std = metric_stds['eval_exact_match_for_{}'.format(classification)]
        exact_match[classification].append(str(metric_mean) + "\u00B1" + str(metric_std))
    
    for non_classification in non_classifications:
        metric_mean = metric_means['eval_rougeL_for_{}'.format(non_classification)]
        metric_std = metric_stds['eval_rougeL_for_{}'.format(non_classification)]
        rougeL[non_classification].append(str(metric_mean) + "\u00B1" + str(metric_std))
    exact_match['eval_exact_match'].append(str(metric_means['eval_exact_match']) + "\u00B1" + str(metric_stds['eval_exact_match']))
    rougeL['eval_rougeL'].append(str(metric_means['eval_rougeL']) + "\u00B1" + str(metric_stds['eval_rougeL']))
english_rougeL = pd.DataFrame(rougeL)
english_rougeL.index = english_exp_names_1 + english_exp_names_3
english_exact_match = pd.DataFrame(exact_match)
english_exact_match.index = english_exp_names_1 + english_exp_names_3

def generate_new_columns(columns):
    new_columns = []
    for c in columns:
        new_column = []
        for first in c.split('_'):
            new_column.append(first[0].upper())
        new_columns.append("".join(new_column))
    return new_columns
print(english_rougeL.columns)
english_exact_match.columns = generate_new_columns(english_exact_match.columns)
english_rougeL.columns = generate_new_columns(english_rougeL.columns)

Index(['data_to_text', 'grammar_error_correction', 'title_generation',
       'keyword_tagging', 'overlap_extraction', 'question_rewriting',
       'eval_rougeL'],
      dtype='object')


In [2]:
english_exact_match.columns

Index(['AC', 'CEC', 'CR', 'DAR', 'WA', 'TE', 'EEM'], dtype='object')

In [6]:
english_rougeL

,DTT,GEC,TG,KT,OE,QR,ER
english,43.3±0.5,85.7±1.1,37.4±0.5,64.8±1.9,32.7±1.4,68.0±0.2,54.8±0.4
english_stopword,43.8±0.4,85.2±0.5,37.2±0.3,63.6±0.3,34.2±1.0,68.0±0.3,53.8±0.4
english_delete_5,42.2±0.3,84.1±0.7,37.2±0.3,64.2±1.2,37.4±1.4,67.4±0.2,53.7±0.4
english_delete_10,42.2±0.5,85.0±2.0,36.8±0.2,64.0±1.0,34.4±1.8,67.5±0.0,53.6±0.3
english_insert_5,41.8±0.4,85.3±0.8,37.2±0.2,61.1±2.6,38.5±2.5,67.5±0.5,54.0±0.1
english_insert_10,42.3±0.5,86.6±0.4,37.0±0.2,65.8±1.2,36.7±1.9,67.3±0.3,53.8±0.0
english_replace_5,42.7±0.3,85.1±1.1,37.1±0.4,63.5±2.3,35.0±1.2,67.8±0.2,54.0±0.2
english_replace_10,42.5±0.4,84.0±1.5,36.9±0.4,65.6±1.6,36.0±3.3,68.2±0.5,53.6±0.6
english_repeat_sentences,42.9±0.4,85.9±0.3,37.4±0.3,64.8±1.6,36.8±2.5,67.3±0.3,54.4±0.2
english_shuffle_sentences,41.9±0.3,86.4±0.7,37.1±0.9,64.6±1.2,38.2±1.8,67.6±0.4,54.2±0.2


In [8]:
print(english_exact_match.to_latex())
# print(english_rougeL.to_latex())

\begin{tabular}{llllllll}
\toprule
{} &        AC &       CEC &        CR &       DAR &        WA &        TE &       EEM \\
\midrule
english                     &  63.6±1.2 &  48.2±3.0 &  48.1±0.9 &  57.0±1.2 &  51.8±0.4 &  46.6±0.5 &  36.9±0.5 \\
english\_stopword            &  63.2±0.6 &  49.0±0.5 &  49.1±2.2 &  49.9±1.7 &  50.5±1.9 &  44.5±0.6 &  36.1±0.5 \\
english\_delete\_5            &  60.7±2.0 &  49.1±2.3 &  47.2±0.7 &  53.1±1.0 &  52.8±0.5 &  44.7±1.1 &  36.0±0.6 \\
english\_delete\_10           &  61.1±2.0 &  50.7±1.2 &  49.5±1.3 &  52.9±4.5 &  50.8±1.3 &  44.0±1.4 &  36.0±0.4 \\
english\_insert\_5            &  60.8±1.4 &  48.5±0.4 &  49.5±1.0 &  55.3±1.7 &  51.0±1.2 &  45.7±0.7 &  36.3±0.2 \\
english\_insert\_10           &  60.4±1.4 &  48.7±1.3 &  47.5±0.9 &  56.6±0.9 &  52.5±1.6 &  44.6±0.5 &  36.2±0.2 \\
english\_replace\_5           &  62.0±0.5 &  48.3±0.7 &  48.3±0.4 &  54.4±2.7 &  50.0±0.7 &  45.0±0.8 &  36.0±0.3 \\
english\_replace\_10          &  60.3±0.7 &  49.2±

/tmp/ipykernel_29755/296510610.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(english_exact_match.to_latex())


In [2]:
import json
import collections
import pandas as pd
import numpy as np
xlingual_exp_names_1 = []
xlingual_exp_names_3 = ['xlingual', 'xlingual_stopword', 'xlingual_delete_5', 'xlingual_delete_10', \
        'xlingual_insert_5', 'xlingual_insert_10', \
        'xlingual_replace_5', 'xlingual_replace_10', \
        'xlingual_repeat_sentences', 'xlingual_shuffle_sentences', 'xlingual_shuffle_words', 'xlingual_shuffle_instruction']
exact_match = collections.defaultdict(list)
rougeL = collections.defaultdict(list)
classifications = ['answerability_classification', 'cause_effect_classification', 'textual_entailment']
non_classifications = ['title_generation']
for xlingual_exp_name in xlingual_exp_names_1:
    metric_file = open('/home/gujiashe/Tk-Instruct/tk_outputs/output_{}/all_results.json'.format(xlingual_exp_name), 'r')
    metric = json.load(metric_file)


    for classification in classifications:
        exact_match[classification].append(metric['eval_exact_match_for_{}'.format(classification)])
    exact_match['eval_exact_match'].append(metric['eval_exact_match'])
    for non_classification in non_classifications:
        rougeL[non_classification].append(metric['eval_rougeL_for_{}'.format(non_classification)])
    rougeL['eval_rougeL'].append(metric['eval_rougeL'])

metric_file = open('/home/gujiashe/Tk-Instruct/tk_outputs/output_{}/all_results.json'.format("xlingual"), 'r')
metric = json.load(metric_file)
metric_names = metric.keys()
def create_dict(keys):
    d = {}
    for key in keys:
        d[key] = []
    return d    
for xlingual_exp_name in xlingual_exp_names_3:
    metric_samples = create_dict(metric_names)
    metric_means = create_dict(metric_names)
    metric_stds = create_dict(metric_names)
    for index in [1, 2, 3]:
        _xlingual_exp_name = xlingual_exp_name + "_{}".format(index)

        metric_file = open('/home/gujiashe/Tk-Instruct/tk_outputs/output_{}/all_results.json'.format(_xlingual_exp_name), 'r')
        metric = json.load(metric_file)
        for metric_name in metric_names:
            metric_samples[metric_name].append(metric[metric_name])
    for metric_name in metric_names:
        metric_means[metric_name] = round(np.mean(metric_samples[metric_name]), 1)
    for metric_name in metric_names:
        metric_stds[metric_name] = round(np.std(metric_samples[metric_name]), 1)

    # metrics = metric_means

    for classification in classifications:
        metric_mean = metric_means['eval_exact_match_for_{}'.format(classification)]
        metric_std = metric_stds['eval_exact_match_for_{}'.format(classification)]
        exact_match[classification].append(str(metric_mean) + "\u00B1" + str(metric_std))
    
    for non_classification in non_classifications:
        metric_mean = metric_means['eval_rougeL_for_{}'.format(non_classification)]
        metric_std = metric_stds['eval_rougeL_for_{}'.format(non_classification)]
        rougeL[non_classification].append(str(metric_mean) + "\u00B1" + str(metric_std))
    exact_match['eval_exact_match'].append(str(metric_means['eval_exact_match']) + "\u00B1" + str(metric_stds['eval_exact_match']))
    rougeL['eval_rougeL'].append(str(metric_means['eval_rougeL']) + "\u00B1" + str(metric_stds['eval_rougeL']))
xlingual_rougeL = pd.DataFrame(rougeL)
xlingual_rougeL.index = xlingual_exp_names_1 + xlingual_exp_names_3
xlingual_exact_match = pd.DataFrame(exact_match)
xlingual_exact_match.index = xlingual_exp_names_1 + xlingual_exp_names_3


def generate_new_columns(columns):
    new_columns = []
    for c in columns:
        new_column = []
        for first in c.split('_'):
            new_column.append(first[0].upper())
        new_columns.append("".join(new_column))
    return new_columns
xlingual_exact_match.columns = generate_new_columns(xlingual_exact_match.columns)
xlingual_rougeL.columns = generate_new_columns(xlingual_rougeL.columns)

In [4]:
print(xlingual_exact_match.to_latex())
# print(xlingual_rougeL.to_latex())

\begin{tabular}{lllll}
\toprule
{} &        AC &       CEC &        TE &       EEM \\
\midrule
xlingual                     &  72.0±0.8 &  54.2±0.6 &  20.0±1.2 &  50.2±0.5 \\
xlingual\_stopword            &  69.7±1.9 &  54.8±0.2 &  19.0±1.4 &  50.6±0.1 \\
xlingual\_delete\_5            &  74.0±1.4 &  55.0±0.5 &  18.0±2.5 &  50.9±0.5 \\
xlingual\_delete\_10           &  73.0±2.9 &  54.6±0.6 &  21.0±1.9 &  50.7±0.5 \\
xlingual\_insert\_5            &  70.3±5.2 &  54.4±0.6 &  18.4±1.7 &  50.3±0.4 \\
xlingual\_insert\_10           &  69.3±4.1 &  55.7±0.3 &  19.6±0.6 &  51.4±0.3 \\
xlingual\_replace\_5           &  69.3±4.1 &  54.2±1.3 &  20.8±0.3 &  50.2±1.0 \\
xlingual\_replace\_10          &  67.0±7.9 &  54.2±0.3 &  17.4±0.8 &  49.9±0.1 \\
xlingual\_repeat\_sentences    &  74.7±1.2 &  55.2±0.6 &  20.9±2.4 &  51.3±0.4 \\
xlingual\_shuffle\_sentences   &  67.3±1.2 &  54.9±0.5 &  19.1±1.6 &  50.6±0.6 \\
xlingual\_shuffle\_words       &  70.0±5.7 &  53.2±0.1 &  16.0±2.6 &  49.0±0.2 \\
xlingu

/tmp/ipykernel_35720/285899341.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(xlingual_exact_match.to_latex())


In [56]:
metrics = {'english_exact_match': english_exact_match, 'english_rougeL': english_rougeL, \
    'xlingual_exact_match': xlingual_exact_match, 'xlingual_rougeL': xlingual_rougeL}
for metric_name, metric_var in metrics.items():
    metric_var.to_csv(metric_name+'_means.csv')


In [6]:
import json
import collections
import pandas as pd
import numpy as np
english_exp_names_1 = []
english_exp_names_3 = ['english_new_instruction']

exact_match = collections.defaultdict(list)
rougeL = collections.defaultdict(list)
classifications = ['answerability_classification', 'cause_effect_classification', 'coreference_resolution', \
    'dialogue_act_recognition', 'word_analogy', 'textual_entailment']
non_classifications = ['data_to_text', 'grammar_error_correction', 'title_generation',\
    'keyword_tagging', 'overlap_extraction', 'question_rewriting']
for english_exp_name in english_exp_names_1:
    metric_file = open('/home/gujiashe/Tk-Instruct/tk_outputs/output_{}/all_results.json'.format(english_exp_name), 'r')
    metric = json.load(metric_file)


    for classification in classifications:
        exact_match[classification].append(metric['eval_exact_match_for_{}'.format(classification)])
    exact_match['eval_exact_match'].append(metric['eval_exact_match'])
    for non_classification in non_classifications:
        rougeL[non_classification].append(metric['eval_rougeL_for_{}'.format(non_classification)])
    rougeL['eval_rougeL'].append(metric['eval_rougeL'])

metric_file = open('/home/gujiashe/Tk-Instruct/tk_outputs/output_{}/all_results.json'.format("english_original_instruction"), 'r')
metric = json.load(metric_file)
metric_names = metric.keys()
# print(metric)
def create_dict(keys):
    d = {}
    for key in keys:
        d[key] = []
    return d    
for english_exp_name in english_exp_names_3:
    metric_samples = create_dict(metric_names)
    metric_means = create_dict(metric_names)
    metric_stds = create_dict(metric_names)
    for index in [1, 2, 3]:
        _english_exp_name = english_exp_name + "_{}".format(index)

        metric_file = open('/home/gujiashe/Tk-Instruct/tk_outputs/output_{}/all_results.json'.format(_english_exp_name), 'r')
        metric = json.load(metric_file)
        for metric_name in metric:
            if metric_name.endswith(f'_new_{index}'):
                metric_name_pruned = metric_name.replace(f'_new_{index}', '')
            else:
                metric_name_pruned = metric_name
            metric_samples[metric_name_pruned].append(metric[metric_name])
    
    for metric_name in metric_names:
        metric_means[metric_name] = round(np.mean(metric_samples[metric_name]), 1)
    for metric_name in metric_names:
        metric_stds[metric_name] = round(np.std(metric_samples[metric_name]), 1)

    # metrics = metric_means

    for classification in classifications:
        metric_mean = metric_means['eval_exact_match_for_{}'.format(classification)]
        metric_std = metric_stds['eval_exact_match_for_{}'.format(classification)]
        exact_match[classification].append(str(metric_mean) + "\u00B1" + str(metric_std))
    
    for non_classification in non_classifications:
        metric_mean = metric_means['eval_rougeL_for_{}'.format(non_classification)]
        metric_std = metric_stds['eval_rougeL_for_{}'.format(non_classification)]
        rougeL[non_classification].append(str(metric_mean) + "\u00B1" + str(metric_std))
    exact_match['eval_exact_match'].append(str(metric_means['eval_exact_match']) + "\u00B1" + str(metric_stds['eval_exact_match']))
    rougeL['eval_rougeL'].append(str(metric_means['eval_rougeL']) + "\u00B1" + str(metric_stds['eval_rougeL']))
english_rougeL = pd.DataFrame(rougeL)
english_rougeL.index = english_exp_names_1 + english_exp_names_3
english_exact_match = pd.DataFrame(exact_match)
english_exact_match.index = english_exp_names_1 + english_exp_names_3

def generate_new_columns(columns):
    new_columns = []
    for c in columns:
        new_column = []
        for first in c.split('_'):
            new_column.append(first[0].upper())
        new_columns.append("".join(new_column))
    return new_columns
english_exact_match.columns = generate_new_columns(english_exact_match.columns)
english_rougeL.columns = generate_new_columns(english_rougeL.columns)

In [8]:
# print(english_exact_match.to_latex())
print(english_rougeL.to_latex())

\begin{tabular}{llllllll}
\toprule
{} &       DTT &       GEC &        TG &        KT &        OE &        QR &        ER \\
\midrule
english\_new\_instruction &  44.2±0.3 &  84.3±0.3 &  24.6±0.2 &  72.0±0.6 &  32.7±0.1 &  81.3±0.2 &  58.0±0.5 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_35720/1850941337.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(english_rougeL.to_latex())


In [4]:
english_exact_match

,AC,CEC,CR,DAR,WA,TE,EEM
english_new_instruction,67.3±1.0,69.3±0.2,56.8±1.6,43.3±2.7,43.2±1.9,52.7±5.8,37.1±0.5


In [3]:
english_rougeL

,DTT,GEC,TG,KT,OE,QR,ER
english_new_instruction,44.2±0.3,84.3±0.3,24.6±0.2,72.0±0.6,32.7±0.1,81.3±0.2,58.0±0.5


## test tk_instruct

In [7]:
import random
def random_shuffle(Definition):

    token_list = Definition.split()
    random.shuffle(token_list)
    return " ".join(token_list)
random_shuffle("I am a student")

'a am student I'

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("allenai/tk-instruct-3b-def")
model = AutoModelForSeq2SeqLM.from_pretrained("allenai/tk-instruct-3b-def")


In [18]:
input_ids = tokenizer.encode(
        # "Definition: return the currency of the country. Now complete the following example - Input: China. Output:", 
        "فتوسنتز فرایندی زیست‌شیمیایی است که در آن، انرژی نورانی خورشید توسط گیاهان و برخی از باکتری‌ها به انرژی شیمیایی ذخیره‌شده در مواد غذایی آن‌ها تبدیل می‌شود. کمابیش همهٔ جانداران روی زمین به آن وابسته‌اند. در عمل فتوسنتز، اندام‌هایی مانند برگ که دارای سبزینه هستند، کربن دی‌اکسید، آب و نور را جذب کرده و به کلروپلاست می‌رسانند. طی واکنش‌هایی که درون کلروپلاست انجام می‌گیرد، این مواد به اکسیژن و کربوهیدرات‌ها تبدیل می‌شوند. همه اکسیژن کنونی موجود بر روی زمین، فراوردهٔ فتوسنتز است. برخی از کربوهیدرات‌های مهم تولیدشده مانند گلوکز، می‌توانند به دیگر مواد آلی، لیپیدها، نشاسته، سلولز و پروتئین تبدیل شوند که برای تبدیل‌شدن به پروتئین، نیاز به نیتروژن دارند. ژان باپتیست ون هلمونت، یکی از نخستین آزمایش‌های مربوط به فتوسنتز را انجام داد. همه بخش‌های سبزرنگ گیاه، قادر به انجام عمل فتوسنتز هستند. مادهٔ سبز موجود در گیاهان که سبزینه یا کلروفیل نام دارد، آغازکنندهٔ واکنش‌های فتوسنتز است. فتوسنتز در اندام‌هایی که فاقد سبزینه هستند، انجام نمی‌گیرد.",
        return_tensors="pt"
    )
output = model.generate(input_ids, max_length=10)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)

## load ni_dataset_crud

In [27]:
from datasets import load_dataset, load_metric
from transformers import set_seed
set_seed(0)
raw_datasets = load_dataset(
    "../src/induction_dataset.py", 
    data_dir='../induction_data/', 
    task_dir='../induction_data/tasks/', 
    cache_dir='../cache/',
    max_num_instances_per_task=100,
    max_num_instances_per_eval_task=100,
    download_mode = 'reuse_cache_if_exists'
)
print(raw_datasets['test'][1])

Using custom data configuration default-ef21323b704ac40a


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset natural_instructions downloaded and prepared to ../cache/natural_instructions/default-ef21323b704ac40a/2.0.0/3cc8a767ebfc2ec331ee88376c2925a8f55abffc944bdfeacb4be90f468c4b97. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'Task': 'translation_en-es_0', 'Contributors': 'instruction_induction', 'Source': ['instruction_induction'], 'URL': ['instruction_induction'], 'Categories': ['instruction_induction'], 'Reasoning': ['instruction_induction'], 'Definition': ['Translate to Spanish'], 'Positive Examples': [], 'Negative Examples': [], 'Input_language': ['English'], 'Output_language': ['English'], 'Instruction_language': ['English'], 'Domains': ['instruction_induction'], 'Instance': {'input': 'ile', 'output': ['montón']}}


In [3]:
from datasets import load_dataset, load_metric
from transformers import set_seed
set_seed(0)
raw_datasets = load_dataset(
    "../src/ni_dataset.py", 
    data_dir='../data/splits/default', 
    task_dir='../data/tasks', 
    # cache_dir='../cache/',
    max_num_instances_per_task=100,
    max_num_instances_per_eval_task=100,
    # download_mode = 'reuse_cache_if_exists'
    download_mode="force_redownload"
)
print(raw_datasets['validation'][101]['Definition'])

Using custom data configuration default-8e82d9b055b27d6b


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset natural_instructions downloaded and prepared to /home/gujiashe/.cache/huggingface/datasets/natural_instructions/default-8e82d9b055b27d6b/2.0.0/c0040095172c0d76abd173b8640d8a5c5293cbc538c41e16bb88bdb4d0bb6a46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

["Please fill out the bank (_) in the given text with an accurate pronoun regarding the target name between  ** **. The generated word should be a case-sensitive pronoun that matches its position in the text. You can choose from the following pronouns: 'her', 'him', 'he', 'she', and 'his'. "]


In [7]:
print(raw_datasets['test'][101]['Definition'])

["Read the passage and find the corresponding pronoun for the given name. The pronoun should match the given blank(_). The word between ** ** is the target name. The pronoun should be one of 'her', 'him', 'he', 'she' and 'his' with proper casing based on the position in the passage."]


In [ ]:
from datasets import load_dataset, load_metric
from transformers import set_seed
set_seed(0)
raw_datasets = load_dataset(
    "../src/ni_dataset_crud.py", 
    data_dir='../data/splits/default', 
    task_dir='../data/tasks', 
    cache_dir='../cache/',
    max_num_instances_per_task=100,
    max_num_instances_per_eval_task=100,
    download_mode = 'reuse_cache_if_exists'
)
print(raw_datasets['validation'][101]['Definition'])

['In this task your given two statements in Estonian. You must judge whether the second sentence is the cause or effect of the first one. Label the instances as "cause" or "effect" based on your judgment. The sentences are separated by a newline character.']

## Data Aug

In [1]:
import numpy as np
import spacy
import random
from typing import Dict
from tqdm import tqdm
import transformers
from transformers import BertTokenizer, BertModel, BertForMaskedLM

transformers.logging.set_verbosity_error()


In [2]:
class DataAugmentation:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
        self.en = spacy.load('en_core_web_sm')
        

    def delete_num(self, Definition, num=5):

        splited_definition = Definition.split()
        index = [i for i in range(len(splited_definition))]
        index = random.sample(index, len(splited_definition)-num)
        index.sort()
        Definition_crud = " ".join([splited_definition[i] for i in index])
        return Definition_crud
        
    def delete_stopwords(self, Definition):

        #loading the english language small model of spacy
        
        stopwords = self.en.Defaults.stop_words
        lst=[]
        for token in Definition.split():
            if token.lower() not in stopwords:    #checking whether the word is not 
                lst.append(token)                    #present in the stopword list.
        Definition_crud = " ".join(lst)     
        return Definition_crud

    def insert_mask(self, Definition, num_mask=5):

        token_list = Definition.split()


        num = 0
        while num < num_mask:
            num += 1
            insert_position = random.randint(1, len(token_list) - 1)
            token_list.insert(insert_position, '[MASK]')
    

        input_txt = ' '.join(token_list)

        inputs = self.tokenizer(input_txt, return_tensors='pt')

        
        input_ids = inputs['input_ids'][0].numpy()
        if input_ids.shape[0]>512:
            return Definition
        outputs = self.model(**inputs)
        predictions = outputs[0]

        _, sorted_idx = predictions[0].sort(dim=-1, descending=True)
        
        for k in range(1):
            predicted_index = [sorted_idx[i, k].item() for i in range(0, len(predictions[0])-1)]
            predicted_token = []
            for x in range(1, len(predictions[0])-1):
                if input_ids[x] == 103:
                    predicted_token.append(self.tokenizer.convert_ids_to_tokens([predicted_index[x]])[0])
        copy_token = predicted_token.copy()
        token_list_copy = token_list.copy()
        for i, token in enumerate(token_list):
            if token == '[MASK]':
                if len(predicted_token)==0:
                    print(Definition)
                    print(token_list_copy)
                    print(copy_token)
                token_list[i] = predicted_token.pop(0)
        final_tokens = []
        for token in token_list:
            if token.startswith('##'):
                final_tokens[-1] = final_tokens[-1] + token[2:]
            else:
                final_tokens.append(token)

        return " ".join(final_tokens)

    def replace_num(self, Definition, num_mask=5):

        inputs = self.tokenizer(Definition, return_tensors='pt')
        input_ids = inputs['input_ids'][0]

        index = [i for i in range(len(input_ids))]

        index = random.sample(index, num_mask)

        index.sort()
        for i in index:
            input_ids[i] = 103


        if len(input_ids)>512:
            return Definition
        inputs['input_ids'][0] = input_ids
        outputs = self.model(**inputs)
        predictions = outputs[0]

        _, sorted_idx = predictions[0].sort(dim=-1, descending=True)

        for k in range(1):
            predicted_index = [sorted_idx[i, k].item() for i in range(0, len(predictions[0])-1)]
            for x in range(1, len(predictions[0])-1):
                if input_ids[x] == 103:
                    input_ids[x] = predicted_index[x]

        return self.tokenizer.decode(input_ids[1: -1])
    
    def shuffle_words(self, Definition):

        token_list = Definition.split()
        random.shuffle(token_list)
        return " ".join(token_list)
    
    def shuffle_sentences(self, Definition):

        doc = self.en(Definition)
        sents = list(map(str, doc.sents))
        random.shuffle(sents)
        return " ".join(sents)

    def repeat_sentences(self, Definition, index = None):
        doc = self.en(Definition)
        sents = list(map(str, doc.sents))
        if None == index:
            index = random.randint(0, len(sents)-1)
        sents = sents[:index] + [sents[index]] + sents[index:]
        return " ".join(sents)

In [3]:
original_instruction = "Given an incorrect English sentence. Your task is to correct the input sentence and write out his correct form."

data_augmentation = DataAugmentation()
data_crud= data_augmentation.repeat_sentences(original_instruction, index = 0)
print(data_crud)

Given an incorrect English sentence. Given an incorrect English sentence. Your task is to correct the input sentence and write out his correct form.


In [11]:
import inspect
attrs = (getattr(data_augmentation, name) for name in dir(data_augmentation))
methods = filter(inspect.ismethod, attrs)
compares = [["Original Instruction", original_instruction]]
for method in methods:
    if method.__name__ != '__init__':
        compares.append([method.__name__, method(original_instruction)])

In [46]:
df_compares = pd.DataFrame(compares)
df_compares.columns = ["Method", "Perturbed Instruction"]
df_compares.index = df_compares["Method"]

df_compares.iloc[:, 1:]

,Perturbed Instruction
Method,
Original Instruction,Given an incorrect English sentence. Your task...
delete_num,Given an English sentence. Your task is to cor...
delete_stopwords,Given incorrect English sentence. task correct...
insert_mask,Given is an incorrect English - input sentence...
repeat_sentences,Given an incorrect English sentence. Your task...
replace_num,Given an incorrect English translation. Your t...
shuffle_sentences,Your task is to correct the input sentence and...
shuffle_words,is correct form. Given write sentence. his You...


In [41]:
import pandas as pd
print(df_compares.style.to_latex(hrules=True))


\begin{tabular}{ll}
\toprule
 & Perturbed Instruction \\
Method &  \\
\midrule
Original Instruction & Given an incorrect English sentence. Your task is to correct the input sentence and write out his correct form. \\
delete_num & Given an English sentence. Your task is to correct the input sentence his correct \\
delete_stopwords & Given incorrect English sentence. task correct input sentence write correct form. \\
insert_mask & Given is an incorrect English - input sentence. " Your task is to correct the input sentence and to write out his correct form. \\
repeat_sentences & Given an incorrect English sentence. Your task is to correct the input sentence and write out his correct form. Your task is to correct the input sentence and write out his correct form. \\
replace_num & Given an incorrect English translation. Your task is to read the English, and write out his correct form. \\
shuffle_sentences & Your task is to correct the input sentence and write out his correct form. Given an 

In [49]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer.decode(10434)

'2 0 1 8'

In [26]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
tokenizer.tokenize(dataset)

['It', 'was', 'a', 'beautiful', 'day', ',', 'perfect', 'for', 'a', 'trip', '.']

## fill_mask

In [29]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
 
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
input_txt = data_crud
inputs = tokenizer(input_txt, return_tensors='pt', truncation=True, max_length=512)

In [30]:
input_ids = inputs['input_ids'][0].numpy()

In [ ]:
model = BertForMaskedLM.from_pretrained('bert-base-cased')
 
outputs = model(**inputs)
predictions = outputs[0]

In [37]:
_, sorted_idx = predictions[0].sort(dim=-1, descending=True)
input_ids = inputs['input_ids'][0].numpy()
for k in range(1):
    predicted_index = [sorted_idx[i, k].item() for i in range(0, len(predictions[0])-1)]
    print(predicted_index)
    predicted_token = []
    for x in range(1, len(predictions[0])-1):
        if input_ids[x] == 103:
            predicted_token.append(tokenizer.convert_ids_to_tokens([predicted_index[x]])[0])
        else:
            predicted_token.append(tokenizer.convert_ids_to_tokens([input_ids[x]])[0])
    print(predicted_token)

[119, 119, 1541, 1108, 1216, 170, 2712, 1285, 117, 1103, 3264, 1285, 1111, 170, 1263, 3868, 119]
['It', 'really', 'was', 'such', 'a', 'beautiful', 'day', ',', 'the', 'perfect', 'day', 'for', 'a', 'long', 'trip', '.']


In [36]:
tokens = []
for token in predicted_token:
    if token.startswith('##'):
        tokens[-1] = tokens[-1] + token[2:]
    else:
        tokens.append(token)
print(" ".join(tokens))

It really was such a beautiful day , the perfect day for a long trip .


## Shuffle sentences 

In [6]:
import spacy
import random
nlp = spacy.load("en_core_web_sm")

In [7]:
doc = nlp("Read the passage and find the corresponding pronoun for the given name. The pronoun should match the given blank(_). The word between ** ** is the target name. The pronoun should be one of 'her', 'him', 'he', 'she' and 'his' with proper casing based on the position in the passage.")

In [29]:
sents = list(map(str, doc.sents))
random.shuffle(sents)
sents

["The pronoun should be one of 'her', 'him', 'he', 'she' and 'his' with proper casing based on the position in the passage.",
 'Read the passage and find the corresponding pronoun for the given name.',
 'The pronoun should match the given blank(_).',
 'The word between ** ** is the target name.']

In [27]:
type(sents[0])

str

In [10]:
for sent in list(doc.sents):
    print(sent)

Read the passage and find the corresponding pronoun for the given name.
The pronoun should match the given blank(_).
The word between ** ** is the target name.
The pronoun should be one of 'her', 'him', 'he', 'she' and 'his' with proper casing based on the position in the passage.


## Category

In [7]:
import os
import json
import collections
c = collections.defaultdict(int)
with open("/home/gujiashe/Tk-Instruct/data/splits/xlingual/test_tasks.txt", encoding="utf-8") as split_f:
    for line in split_f:
        task_name = line.strip()
        task_path = os.path.join("/home/gujiashe/Tk-Instruct/data/tasks", task_name + ".json")
        
        with open(task_path, encoding="utf-8") as task_f:
            s = task_f.read()
            task_data = json.loads(s)
            print(task_data["Categories"])
            c[task_data["Categories"][0]] += 1
print(c)

['Title Generation']
['Cause Effect Classification']
['Textual Entailment']
['Cause Effect Classification']
['Answerability Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Cause Effect Classification']
['Textual Entailment']
['Cause Effect Classification']
['Ca